In [ ]:
RAY_SWITCH = True # allow task-based parallel optimization by ray if set to True  

In [ ]:
from sympy import init_printing; init_printing()

if RAY_SWITCH: 
    import ray; ray.init()
    def raylize_func(func):
        @ray.remote
        def raylized_func(*args, **kwargs):
            return func(*args, **kwargs)    
        return raylized_func

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # display all expression in one cell instead of the last one

## Initialization of Variables

In [ ]:
from sympy import integrate, Eq, symbols, Array, Function
from sinupy.algebra.tensor import cross
x, y, z = symbols('x, y, z', real=True); t = symbols("t", real=True)
E_x, E_y, E_z = symbols("E_x, E_y, E_z", real=True); E = Array([E_x, E_y, E_z]); E # Array([E_x, E_y, E_z]) or Array([E_x, 0, 0])
B_x, B_y, B_z = symbols("B_x, B_y, B_z", positive=True); B = Array([0, 0, B_z]); B # Array([B_x, B_y, B_z]) or Array([0, 0, B_z])
r_x, r_y, r_z = [f(t) for f in symbols("r_x, r_y, r_z", real=True, cls=Function)]; r = Array([r_x, r_y, r_z])
v_x, v_y, v_z = [f(t) for f in symbols("v_x, v_y, v_z", real=True, cls=Function)]; v = Array([v_x, v_y, v_z])
m = symbols('m', positive=True); q = symbols('q', nonzero=True, real=True)


## Set up the Newton Second Law Equation (ode system)

In [ ]:
Newton2 = Eq(q*E + q* cross(v, B), m * v.diff(t) ); 
diff_r_is_v = Eq(r.diff(t), v); diff_r_is_v

In [ ]:
from sympy import simplify, sqrt, trigsimp, refine, Q, sin, cos, pi, Abs
simplify = raylize_func(simplify)
trigsimp = raylize_func(trigsimp)
# [func = raylize_func(func) for func in [simplify,sqrt]]
import sympy.solvers.ode as ode
from wagglepy.symutil import divide_Array_Eq

# v_sol = ode.systems.dsolve_system(
#     divide_Array_Eq(Newton2))
# for i in range(3): v_sol[0][i]
# for i in range(3): v_sol[0][i].simplify()

v_r_sol = ode.systems.dsolve_system(
    [*divide_Array_Eq(Newton2),
     *divide_Array_Eq(diff_r_is_v)])
for comp_sol in v_r_sol[0]: comp_sol

v_r_sol[0] = ray.get([simplify.remote(comp_sol) for comp_sol in v_r_sol[0]]) if RAY_SWITCH \
    else [simplify(comp_sol) for comp_sol in v_r_sol[0]]
    
for comp_sol in v_r_sol[0]: comp_sol

### Time Averaging

In [ ]:
from wagglepy.symutil.characteristics import min_period
from wagglepy.symutil.op import time_average_of_TrigFunc
tprime = symbols('t^{\prime}', real=True)
T = min_period(v_r_sol[0][0].rhs, t)


def subs_multiple_pi_trig_func(expr):
    return expr\
            .subs(sin((2*pi*q)/Abs(q)), 0)\
            .subs(sin((pi*q)/Abs(q)), 0)\
            .subs(cos((2*pi*q)/Abs(q)), 1)
def t_avg_Eq(eq, Tmin, Tmax):
        return Eq(
            time_average_of_TrigFunc(eq.lhs, t, T=[Tmin, Tmax]),
            time_average_of_TrigFunc(eq.rhs, t, T=[Tmin, Tmax]))
def simplify_rhs(eq):
    return Eq(eq.lhs, eq.rhs.simplify())

if RAY_SWITCH: 
    t_avg_Eq = raylize_func(t_avg_Eq)
    subs_multiple_pi_trig_func = raylize_func(subs_multiple_pi_trig_func)
    simplify_rhs = raylize_func(simplify_rhs)
    
    # t_avg_EqSol.remote(v_r_sol[0][0])
    # t_avg_EqSol.remote(v_r_sol[0][1])
    # t_avg_EqSol.remote(v_r_sol[0][2])
    # print("The above formulas might look too cumbersome, let's simplify them.")
    futures = []
    for i, comp_sol in enumerate(v_r_sol[0]):
        t_avg_comp_future = t_avg_Eq.remote(comp_sol, 0, T) if i < 3 \
            else t_avg_Eq.remote(comp_sol, tprime, tprime+T)
        futures.append(
                simplify_rhs.remote(
                    subs_multiple_pi_trig_func.remote(
                        trigsimp.remote(
                                subs_multiple_pi_trig_func.remote(t_avg_comp_future), 
                        method="fu")
                    )
                ))

    t_avg_Eq_list = ray.get(futures)
    for comp_sol in t_avg_Eq_list: comp_sol
else:
    pass


## Visualization of The Electron Trace Interacting with a Static E.M. Field 

In [ ]:
from scipy import constants
import numpy as np
from sympy import lambdify, Symbol
v_r_sol_N = [comp_sol.rhs\
             .subs([(E_x, 6e5), (E_y, 6e5), (E_z, 1e4)])\
             .subs([(B_x, 0.0), (B_y, 0.0), (B_z, 5e-2)])\
             .subs([('C1', 1e3), ('C2', 0.0), ('C3', 0.0), ('C4', 0.0), ('C5', 0.0), ('C6', 0.0)])\
             .subs(q, constants.e).subs(m, constants.m_p) for comp_sol in v_r_sol[0]]
for i in range(6): v_r_sol_N[i]

In [ ]:
t1 = 2e-4
dt = 6e-8
num_of_t = int(t1 / dt); num_of_t
t_arr = np.arange(num_of_t) * dt

x_arr, y_arr, z_arr = (lambdify(t, v_r_sol_N[i], 'numpy')(t_arr) for i in [3, 4, 5])
# x_rest_arr = x_arr - a_0**2 / 4 * (t_arr-x_arr)
# z_arr = np.zeros_like(x_arr)

vx_arr, vy_arr, vz_arr = (lambdify(t, v_r_sol_N[i], 'numpy')(t_arr) for i in [0, 1, 2])
v_arr = np.sqrt(vx_arr**2 + vy_arr**2 + vz_arr**2)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter3d(
    x=x_arr, y=y_arr, z=z_arr,
    marker=dict(
        size=2,
        color=v_arr,
        colorscale='Viridis',
    ),
    line=dict(
        width=2,
        color=v_arr,
        colorscale='Viridis',
    )
))

fig.update_layout(
    scene=dict(
        aspectratio = dict( x=1, y=1, z=1.0 ),
        aspectmode = 'data'))
# fig.show()


## Bugs to be reported to Sympy

In [ ]:
from sympy import sin, sinh, pi, symbols
a = symbols("a", positive=True)
b, t = symbols("b, t", real=True)
sin(a*t).period(t)

In [ ]:
sin(-a*t).period(t) # AttributeError: 'Mul' object has no attribute 'period'
sin((-1)*a*t).period(t) # AttributeError: 'Mul' object has no attribute 'period'

In [ ]:
sin(b*t).period(t)